In [ ]:
import boto3
from datetime import datetime
import pandas as pd

### ==============================================================================================

In [ ]:
env = 'dev'
CACHED_S3_BUCKET = 'saiva-dev-data-bucket'
client = 'trio'
task_name = 'saiva-3-day-hosp-v1'
facilities = [1, 7, 21, 42, 52, 55, 186, 194, 265, 273, 274, 275, 276, 277, 278, 279]

In [ ]:
env = 'staging'
CACHED_S3_BUCKET = 'saiva-staging-data-bucket'
client = 'avante'
task_name = 'saiva-3-day-hosp-v1'
# facilities = [1, 3, 4, 5, 6, 7, 8, 9, 10, 13, 21]
facilities = [7]

In [ ]:
env = 'prod'
CACHED_S3_BUCKET = 'saiva-prod-data-bucket'
client = 'dycora'
task_name = 'dycora-3-day-hosp'
# facilities = [121, 302]
facilities = [302]

## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
client = 'trio'
env = 'dev'
# facilities = [1,3,4,5,6,7,8,9,10,13,21] avante
facilities = [279,278,277,276,275,274,273,194,1,21,186,265,55,52,42,7]  # trio
CACHED_S3_BUCKET = 'saiva-dev-data-bucket'
task_name = 'saiva-3-day-hosp-v2'
prediction_start_date = '2020-12-01'
prediction_end_date = '2021-01-31'

saiva = boto3.session.Session()
boto3.setup_default_session()

In [ ]:
vpn_instance = {
    "dev": "i-07197a52885c34d68",
    "staging": "i-07d9c5587e7637960",
    "prod": "i-0e5f01117bb50fdb1"
}

AutoScalingGroupName = {
    "dev": "dev-Internal-ASG",
    "staging": "staging-Internal-ASG",
    "prod": "prod-Internal-ASG"
}

ClusterName = {
    "dev": "dev-Internal",
    "staging": "staging-Internal",
    "prod": "prod-Internal"  
}

networkConfiguration = {
    "dev": {
        'awsvpcConfiguration': {
            'securityGroups': ['sg-008159c0210f49f96'], 
            'subnets': ['subnet-0eb41526eff222cab', 'subnet-04b6e7f553bd31909'], 
            'assignPublicIp': 'DISABLED'}},
    "staging": {
        'awsvpcConfiguration': {
            'securityGroups': ['sg-0f8d382201750604e'], 
            'subnets': ['subnet-0b9343ae7c5712608', 'subnet-01d3901613027f4d9'], 
            'assignPublicIp': 'DISABLED'}},
    "prod": {
            "awsvpcConfiguration": {
                "securityGroups": ["sg-041a9f4c8e2f4fd56"],
                "subnets": ["subnet-0c87ee56c3e4e27d1", "subnet-065f42df2509bd9f5"],
                "assignPublicIp": "DISABLED",
            }
        }
}

In [ ]:
# Turn on the Avante VPN Server 

ec2 = boto3.client('ec2', region_name='us-east-1')
ec2.start_instances(
    InstanceIds=[
        vpn_instance[env], # Instance ID of the VPN Server
    ]
)

In [ ]:
asg = boto3.client('autoscaling', region_name='us-east-1')

In [ ]:
desired_count = len(facilities)
# Scale Up the ASG
response = asg.update_auto_scaling_group(
        AutoScalingGroupName=AutoScalingGroupName[env],
        DesiredCapacity=desired_count,
        MaxSize=desired_count,
        DefaultCooldown=100
    )
response

In [ ]:
# List all running services or Tasks

ecs = boto3.client('ecs', region_name='us-east-1')
response = ecs.list_tasks(cluster="dev-Internal",maxResults=100,launchType='EC2')
response['taskArns']

In [ ]:
def start_backfill(facility_id, prediction_start_date,prediction_end_date):
    response = ecs.run_task(
        taskDefinition=f"{env}-{task_name}",
        cluster=f'{env}-Internal',
        overrides={
            "containerOverrides": [
                {
                    "name": f"{task_name}",
                    "command": [
                        "/bin/bash",
                        "-c",
                        f"python /src/run_backfill.py --client {client} --facilityids '[{facility_id}]' --prediction_start_date '{prediction_start_date}' --prediction_end_date '{prediction_end_date}' --s3-bucket {CACHED_S3_BUCKET} --replace_existing_predictions True ",
                    ],
                }
            ]
        },
        launchType="EC2",
        networkConfiguration=networkConfiguration[env],
        placementStrategy=[
            {
                "field": "instanceId",
                "type": "spread"
            }
        ]
    )
    return response

In [ ]:
print(datetime.now())
results = [start_backfill(
    facility_id=facility, 
    prediction_start_date=prediction_start_date, 
    prediction_end_date=prediction_end_date) for facility in facilities]
results

In [ ]:
# Scale DOWN the ASG.
response = asg.update_auto_scaling_group(
        AutoScalingGroupName=AutoScalingGroupName[env],
        DesiredCapacity=0,
        MaxSize=0,
        DefaultCooldown=100
    )
response